In [1]:
import torch.utils.data as dt
import torch
import os
from PIL import Image
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
to_img = ToPILImage()
import torch
import torch.nn as nn
import torch.utils.data as dt
#from carvana_dataset import CarvanaDataset
#from model import SegmenterModel
from torch.autograd import Variable
import torch.optim as optim
from tensorboardX import SummaryWriter
import os
from tqdm import *
import numpy as np


class CarvanaDataset(dt.Dataset):
    """ 
        Carvana features dataset.  Override torch Dataset class to implements reading from h5 files

    """

    def __init__(self, data_path, mask_path, input_size=224):
        """
        Args:
            data_path (string): Path to the images data files.
            mask_path (string): Path were images masks are placed
        """
        self.files = os.listdir(data_path)
        self.files.sort()
        self.mask_files = os.listdir(mask_path)
        self.mask_files.sort()
        self.data_path = data_path
        self.mask_path = mask_path
        assert (len(self.files) == len(self.mask_files))
        self.input_size = input_size

        self.preprocess = transforms.Compose([
            transforms.Scale((input_size, input_size)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.files)

    def pil_load(self, path, is_input=True):
        # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
        with open(path, 'rb') as f:
            with Image.open(f) as img:
                return img.convert('RGB')

    def pil_save(self, t, img_path):
        a = to_img(t)
        a.save(img_path, 'PNG')

    def __getitem__(self, idx):
        f_name = os.path.join(self.data_path, self.files[idx])
        m_name = os.path.join(self.mask_path, self.mask_files[idx])

        if os.path.exists(f_name) == False:
            raise Exception('Missing file with name ' + f_name + ' in dataset')

        input = self.pil_load(f_name)
        target = self.pil_load(m_name, False)

        input = self.preprocess(input)
        target = self.preprocess(target)
        target = torch.sum(target, dim=0).unsqueeze(0)
        target[ torch.gt(target, 0) ] = 1

        return input, target

In [2]:
log = './log/'
train = './05/practise/train/'
train_masks = './05/practise/train_masks/'
test = './05/practise/test/'
test_masks = './05/practise/test_masks/'



ds = CarvanaDataset(train, train_masks)
ds_test = CarvanaDataset(test, test_masks)

dl      = dt.DataLoader(ds, shuffle=True, num_workers=4, batch_size=5)
dl_test = dt.DataLoader(ds_test, shuffle=False, num_workers=4, batch_size=5)

C:\Users\George\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:207: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [33]:
useCuda = True
n_epoch = 10
log = './log/'

if os.path.exists(log) is False:
    os.mkdir(log)
tb_writer = SummaryWriter(log_dir='log')

if __name__ == '__main__':

    #m = SegmenterModel()

    criterion = nn.CrossEntropyLoss()
    #optimizer = optim.Adam(m.parameters(), lr=0.001)

    if useCuda == True:
        #m = m.cuda()
        criterion = criterion.cuda()

    global_iter = 0
    for epoch in range(0, n_epoch):
        print("Current epoch: ", epoch)
        epoch_loss = 0
        #m.train(True)
        for  (i, t) in ((dl)):
            i = Variable(i)
            t = Variable(t.squeeze_(1).long())
            if useCuda:
                i = i.cuda()
                t = t.cuda()
            o = m(i, useCuda)
            if useCuda: o = o.cuda()
            loss = criterion(o, t)
            loss.backward()
            optimizer.step()

            global_iter += 1
            epoch_loss += loss.data[0]


Current epoch:  0


BrokenPipeError: [Errno 32] Broken pipe